# Trail Camera Image and Video Search with Meta AI ImageBind
This project demonstrates how to build multi-modal search (image and video) using the [Meta AI ImageBind](https://imagebind.metademolab.com/) and [Weaviate](https://weaviate.io/) vector database.

## Weaviate Setup

The ImageBind model is only available with local Weaviate deployments with Docker or Kubernetes.

ImageBind is not supported with Weaviate Cloud Services (WCS).

### Steps to deploy Weaviate locally with ImageBind

1. Install Docker.
    
   > If you are new to `Docker Compose`, [here are instructions on how to install it](https://docs.docker.com/compose/install/).

2. Run Weaviate+Bind with Docker Compose

     In the terminal, navigate to the root director of this project and locate the file `docker-compose.yml` and call:

    ```
    docker compose up
    ```
    
    > Note #1 - the first time you run the command, Docker will download a ~6GB image.
  
    > Note #2 – running this Docker image requires 12GB of RAM.  If you're in Windows you'll need to adjust your .wslconfig to include the following:

    ```
    [wsl2]
    memory=12GB
    ```

### Dependencies

Install the Weaviate client library for Python.

In [ ]:
%pip install weaviate-client

## Configuration

Verify the Weaviate client can connect to the local Weaviate server.



In [ ]:
import weaviate
import os

# Connect to Weaviate
client = weaviate.Client(
  url="http://localhost:8080",  # URL to your local Weaviate instance
)

client.is_ready() # Test the connection

### Create `Media` collection

The collection has the following key characteristics:
1. Class: `"Media"`
2. Vectorizer: `multi2vec-bind`
3. Image property: `"img"` - Weaviate will use values in "img" property to generate vectors. Note, you can call it anything you want.

In [ ]:
# Delete the collection if it exists.
# Note you should skip this step if you don't want to reimport the data every time.
if client.schema.exists("Media"):
    client.schema.delete_class("Media")

media = {
    "classes": [
        {
            "class": "Media",            
            "vectorizer": "multi2vec-bind",            
            "moduleConfig": {
                "multi2vec-bind": {
                    "imageFields": ["image"],
                    "videoFields": ["video"],
                }
            },
            "properties": [
                {
                    "dataType": ["blob"],
                    "name": "image"
                },
                {
                    "dataType": ["blob"],
                    "name": "video"
                }
            ]
        }
    ]
}

client.schema.create(media)
print("Successfully created Media collection.")

### Import Media
For every object, we will store:
* `name` - the file name 
* `path` - path to the file, so that we could display returned images at query time.
* (one of the following) media:
    * `image` - a base64 representation of the image file, Weaviate will use it to generate a vector - see `imageFields`.
    * `video` - a base64 representation of the video file, Weaviate will use it to generate a vector - see `videoFields`.


In [ ]:

import base64

# Helper function to convert a file to base64 representation
def toBase64(path):
    with open(path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

#### Import images

In [ ]:
# List of source images 
source = ["cat1.jpg", "cat2.jpg", "cat3.jpg",
          "dog1.jpg", "dog2.jpg", "dog3.jpg",
          "meerkat1.jpg", "meerkat2.jpg", "meerkat3.jpg"]

client.batch.configure(batch_size=3)  # Load images in batches of 3
with client.batch as batch:

    for name in source:
        print(f"Adding {name}")

        # Build the path to the image file
        path = "./source/image/" + name

        # Object to store in Weaviate
        properties = {
            "name": name,
            "path": path,
            "image": toBase64(path), # Weaviate will use the base64 representation of the file to generate a vector.
            "mediaType": "image"
        }

        # Add the object to Weaviate
        client.batch.add_data_object(properties, "Media")

#### Import Video

In [ ]:
# List of source video 
source = [
    "cat-clean.mp4", "cat-play.mp4",
    "dog-high-five.mp4", "dog-with-stick.mp4",
    "meerkat-dig.mp4", "meerkat-watch.mp4"
]

client.batch.configure(batch_size=1)  # Load images in batches of 1, as these might be big files
with client.batch as batch:

    for name in source:
        print(f"Adding {name}")

        # Build the path to the image file
        path = "./source/video/" + name

        # Object to store in Weaviate
        properties = {
            "name": name,
            "path": path,
            "video": toBase64(path), # Weaviate will use the base64 representation of the file to generate a vector.
            "mediaType": "video"
        }

        # Add the object to Weaviate
        client.batch.add_data_object(properties, "Media")

### Check number of objects in the Media collection

In [ ]:
# Display the number of objects in the Media collection
client.query.aggregate("Media").with_meta_count().do()

## Query examples

In [ ]:
# Helper functions to display results
import json
from IPython.display import Image, Audio, Video

def json_print(data):
    print(json.dumps(data, indent=2))

def display_media(item):
    path = item["path"]

    if(item["mediaType"] == "image"):
        display(Image(path))

    elif(item["mediaType"] == "video"):
        display(Video(path))

### Text to Media search

In [ ]:
# Search for media with "dog with stick", "cat playing with mouse", "dog high five", "puppy"
response = (
    client.query
    .get("Media", "name path mediaType")
    .with_near_text(
        #{"concepts": "dog with stick"}
        {"concepts": "cat playing with mouse"}
        # {"concepts": "dog high five"}
        # {"concepts": "puppy"}
    )
    .with_limit(3)
    .do()
)

# Print results
result = response["data"]["Get"]["Media"]

json_print(result)

# Display the first result
display_media(result[0])

### Image to Media search

In [ ]:
Image('test/test-cat.jpg')

In [ ]:
# Search for images that are similar to the provided image of test-meerkat, test-dog, test-cat
response = (
    client.query
    .get("Media", "name path mediaType")
    .with_near_image(
        # {"image": "./test/test-meerkat.jpg"}, # Use file path as the input for the query
        # {"image": "./test/test-dog.jpg"}, # Use file path as the input for the query
        {"image": "./test/test-cat.jpg"}, # Use file path as the input for the query
    )
    .with_limit(5)
    .do()
)

# Print results
result = response["data"]["Get"]["Media"]
json_print(result)

# Display the first image
display_media(result[0])

### Video to Media search

In [ ]:
Video('test/test-meerkat.mp4')

In [ ]:
# Search for images that are similar to the provided image of test-meerkat, test-dog, test-cat
response = (
    client.query
    .get("Media", "name path mediaType")
    .with_near_video(
        # {"video": "./test/test-dog.mp4"}, # Use file path as the input for the query
        # {"video": "./test/test-cat.mp4"}, # Use file path as the input for the query
        {"video": "./test/test-meerkat.mp4"}, # Use file path as the input for the query
    )
    .with_limit(3)
    .do()
)

# Print results
result = response["data"]["Get"]["Media"]
json_print(result)

# Display the first image
display_media(result[0])